## USE CASE  ROCKET4SALES

## 1: Remplissage des valeurs de l'onglet "Job Title correspondance"

**SOLUTION PROPOSEE:** Nous allons mettre en place une fonction qui cherche une correspondance la plus approximative possible des cellules de la colonne jobTitle dans les cellules de la colonne job. Une fois la correspondance trouvée on rempli la colonne jobcorrespondance avec les elements de la colonne job referent correspondants. On aurait pu le faire avec la fonction RECHERCHEV d'Excel mais elle est moins précise et en plus pour cette cellules cette fonction génère de erreurs.

In [17]:
import pandas as pd
from fuzzywuzzy import fuzz

# Lecture de la feuille  JobTitle Correspondance
references_df = pd.read_excel('Alternance_Data_scientist_Cas1.xlsx', sheet_name='JobTitle Correspondance')

# Lecture de la feuille  Linkedin job URL
matching_df = pd.read_excel('Alternance_Data_scientist_Cas1.xlsx', sheet_name='Linkedin job URL')

# Définir une fonction pour trouver la meilleure correspondance approximative
def find_best_match(job_title):
    best_match = None
    highest_similarity = -1

    for reference_title in references_df['Job']:
        similarity = fuzz.ratio(job_title, reference_title)
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = reference_title

    return best_match





matching_df['JObTitle Correspondance'] = matching_df['jobTitle'].apply(find_best_match)

# Mapper les valeurs correspondantes à partir de base1_df
correspondence_dict = dict(zip(references_df['Job'], references_df['Job référent']))
matching_df['JObTitle Correspondance'] = matching_df['JObTitle Correspondance'].map(correspondence_dict)



/Users/barryalphaoumar/opt/anaconda3/envs/My_jupyter_env/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


La partie suivante, c'est pour verifier qu'on a pas de valeur manquante.

In [32]:
value_counts = matching_df['JObTitle Correspondance'].value_counts()

value_counts

Account Executive           2977
BDR                          732
Consultant                   588
Account Manager              539
Marketing                    531
Manager                      335
Intern                       297
Channel                      293
Director                     289
Ingénieur d'affaires         284
OUT                          260
Key Account Manager          194
Customer Success Manager     181
Sales Operations             171
IT                           140
Out                          129
Pre-Sales                    105
Assitant                      66
Pre Sales                     48
GAM                           45
Growth                        42
HR                            35
Operations                    22
Finance                       20
Channel Manager               18
BDR Manager                   17
Assistant                     13
CRO                           13
Owner                          9
Manager Account Manager        8
Customer S

Importation du dataframe obtenu en csv puis chargement dans le fichier excel.

In [36]:
import csv
matching_df.to_csv('Linkedin job URL 2.csv', sep=';', index=False, quoting=csv.QUOTE_NONNUMERIC)

Les données obtenues à partir de ce dataframe sont dans la nouvelle feuille "Linkedin job URL 2"

## 3: SCRAPING DE WTTJ

In [55]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

webdriver_path = "chromedriver"


# Initialisation du service ChromeDriver
service = Service(webdriver_path)

# Démarrer le navigateur Chrome avec les options spécifiées
driver = webdriver.Chrome(service=service)
Companies=[]
jobTitles=[]
jobLocations=[]
postDates=[]
timestamps=[]
jobLinks=[]
jobTypes=[]

for i in range (1,35):
    url=f"https://www.welcometothejungle.com/fr/jobs?page={i}&query=&groupBy=job&sortBy=mostRecent&aroundQuery=France%2C%20France&refinementList%5Bcontract_type_names.fr%5D%5B%5D=CDD%20%2F%20Temporaire&refinementList%5Bcontract_type_names.fr%5D%5B%5D=CDI&refinementList%5Bcontract_type_names.fr%5D%5B%5D=Stage&refinementList%5Bcontract_type_names.fr%5D%5B%5D=Temps%20partiel&refinementList%5Bcontract_type_names.fr%5D%5B%5D=Alternance&refinementList%5Bsectors_name.fr.Tech%5D%5B%5D=SaaS%20%2F%20Cloud%20Services&refinementList%5Bsectors_name.fr.Tech%5D%5B%5D=Logiciels&refinementList%5Bsectors_name.fr.Tech%5D%5B%5D=Big%20Data&refinementList%5Bsectors_name.fr.Tech%5D%5B%5D=Cybersécurité&refinementList%5Bsectors_name.fr.Tech%5D%5B%5D=Application%20mobile&refinementList%5Bsectors_name.fr.Tech%5D%5B%5D=Intelligence%20artificielle%20%2F%20Machine%20Learning&refinementList%5Bsectors_name.fr.Tech%5D%5B%5D=Objets%20connectés&refinementList%5Bsectors_name.fr.Tech%5D%5B%5D=Robotique&refinementList%5Bsectors_name.fr.Tech%5D%5B%5D=Blockchain&refinementList%5Bsectors_name.fr.Publicité%20%2F%20Marketing%20%2F%20Agence%5D=&refinementList%5Bprofession_name.fr.Business%5D%5B%5D=Business%20Development&refinementList%5Bprofession_name.fr.Business%5D%5B%5D=Commercial&refinementList%5Bprofession_name.fr.Business%5D%5B%5D=Account%20Management&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Blanguage%5D%5B%5D=fr&refinementList%5Blanguage%5D%5B%5D=en&range%5Bexperience_level_minimum%5D%5Bmin%5D=0&range%5Bexperience_level_minimum%5D%5Bmax%5D=6"
    driver.get(url)

    time.sleep(20)
    # Faire défiler la page pour charger toutes les offres d'emploi
    actions = ActionChains(driver)
    while True:
        last_height = driver.execute_script("return document.body.scrollHeight")
        actions.send_keys(Keys.END).perform()
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
    wait = WebDriverWait(driver, 20)

    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    jobs=soup.find_all('li',class_='sc-bXCLTC kkKAOM ais-Hits-list-item')
    
    for job in jobs:
        CompanyName = job.find('span', class_="sc-ERObt gTCEVh sc-6i2fyx-3 eijbZE wui-text").text
        jobTitle = job.find('div',class_="sc-bXCLTC hlqow9-0 helNZg").text
        jobLink = job.find('a')['href']
        jobLocation = job.find('span',class_="sc-68sumg-0 gvkFZv").text
        jobType = job.find('i', attrs={'name': 'contract'}).find_next_sibling().text
        postDate = job.find('time').text
        timestamp = job.find('time')['datetime']
        Companies.append(CompanyName)
        jobTitles.append(jobTitle)
        jobLocations.append(jobLocation)
        postDates.append(postDate)
        timestamps.append(timestamp)
        jobLinks.append(jobLink)
        jobTypes.append(jobType)

In [57]:
donnees_scrap = {'Company name': Companies, 'jobCorrespondance':[None] * len(Companies), 'jobDescription': [None] * len(Companies), 'jobLink':jobLinks,'jobLocation':jobLocations,'jobRef':[None] * len(Companies),'jobTitle':jobTitles,'jobType':jobType,'postDate':postDates,'timestamp':timestamps}

# Convertir le dictionnaire en dataframe

df_scrap = pd.DataFrame(donnees_scrap)

La partie suivante, c'est pour trouver les valeurs de jobCorrespondance en utilisant notre fonction definie plus haut.

In [58]:
df_scrap['jobCorrespondance'] = df_scrap['jobTitle'].apply(find_best_match)
df_scrap['jobCorrespondance'] = df_scrap['jobCorrespondance'].map(correspondence_dict)

In [59]:
df_scrap

,Company name,jobCorrespondance,jobDescription,jobLink,jobLocation,jobRef,jobTitle,jobType,postDate,timestamp
0,Dealt,Channel Manager,None,/fr/companies/dealt/jobs/head-of-partnership_l...,Levallois-Perret,None,Head of Partnership,Alternance,il y a 4 heures,2023-07-01T20:00:00.000+02:00
1,Dealt,Account Executive,None,/fr/companies/dealt/jobs/head-of-provider-serv...,Levallois-Perret,None,Responsable communauté de prestataires,Alternance,il y a 4 heures,2023-07-01T20:00:00.000+02:00
2,Dealt,IT,None,/fr/companies/dealt/jobs/project-manager_leval...,Levallois-Perret,None,Project Manager,Alternance,il y a 4 heures,2023-07-01T20:00:00.000+02:00
3,Stockly,Account Executive,None,/fr/companies/stockly/jobs/cdi-account-executi...,Paris,None,Account Executive - CDI - Paris,Alternance,il y a 5 heures,2023-07-01T19:00:00.000+02:00
4,Galadrim,Account Executive,None,/fr/companies/galadrim/jobs/sales-manager-cdi_...,Paris,None,Sales Manager - CDI,Alternance,il y a 7 heures,2023-07-01T17:00:00.000+02:00
...,...,...,...,...,...,...,...,...,...,...
995,Lepermislibre,BDR,None,/fr/companies/lepermislibre/jobs/stage-busines...,Lyon,None,Business Developer,Alternance,il y a 2 mois,2023-05-30T14:00:00.000+02:00
996,Lepermislibre,Intern,None,/fr/companies/lepermislibre/jobs/stage-sales-b...,Lyon,None,Business Developer - Stage de pré-embauche,Alternance,il y a 2 mois,2023-05-30T14:00:00.000+02:00
997,Empowill,Customer Success Manager,None,/fr/companies/empowill/jobs/csm-customer-succe...,Paris,None,Responsable du succès client - Service client ...,Alternance,il y a 2 mois,2023-05-30T14:00:00.000+02:00
998,Diji,BDR,None,/fr/companies/diji/jobs/business-manager_leval...,Levallois-Perret,None,Business Developer,Alternance,il y a 2 mois,2023-05-30T13:00:00.000+02:00


Les colonnes jobdescription et jobref n'ont pas été scrapées par manque de temps. 

Le df_scrap obtenu doit juste être converti en csv afin d'être exportéé vers la feuille excel "wttj jobs offers".

Cette partie n'a pas été faite car le noyau du notebook a planté et la réexecution du noyau entraînerait une grosse perte de temps.

**Idées d'analyses:** On pourrait visualiser la colonne jobCorrespondance afin de voir la repartition des postes dans les offres qu'on a scrapper ou encore les types de contrats. On pourrait aussi voir s'il ya un lien entre le type de contrat et le poste. 

On pourrait aussi voir quelles entreprises recrutent le plus en ce moment et aussi quel poste est le plus prisé en ce moment.